# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Parts Model' {
        package 'Fake Library' {
            part def Part;
            port def Port;
            connection def Connection;
        }
    
        part 'Power Group' : 'Fake Library'::Part {
            part 'Power Source' : 'Fake Library'::Part {
                port 'Power Out' : 'Fake Library'::Port;
            }
            part 'Power User' : 'Fake Library'::Part {
                port 'Power In' : 'Fake Library'::Port;
            }

            connection powerToUser : 'Fake Library'::Connection connect 'Power Source'::'Power Out' to 'Power User'::'Power In';
        }
    
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as nx
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.client import SysML2Client
from pymbe.query.metamodel_navigator import map_inputs_to_results

from pymbe.interpretation.calc_dependencies import generate_execution_order

import os

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [2]:
helper_client = SysML2Client()

path = "C:\\Users\\bjorn\\Documents\\Git\\pyMBE"
file_name = "\\tests\\data\\Simple Parts Model\\elements.json"

helper_client._load_disk_elements(path + str(file_name))

Create a graph representation of the model and load it into memory.

In [3]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [4]:
shorten_pre_bake = {
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [5]:
part_id = 'a1eb8672-fdc4-446e-a004-861974660152'
port_id = 'f29c8a2e-a7d3-4685-bfbb-3d54b5dd704c'
connection_id = '9739035d-8c51-4eab-8ab9-ed703a21f1c4'

In [6]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [7]:
get_types_for_feature(lpg, '7328c370-26d7-40e4-9a36-c88ef76c7d30')

['f29c8a2e-a7d3-4685-bfbb-3d54b5dd704c']

In [8]:
get_features_typed_by_type(lpg, part_id)

['4f446a59-bc83-4285-aa2b-6c5dd9bd5541',
 '648fd9b3-2a6e-40e8-b3e9-b8ebb407ce07',
 'eb96afae-0f09-4912-861e-705bb33a4202']

In [9]:
roll_up_multiplicity_for_type(lpg, helper_client.elements_by_id[connection_id], 'upper')

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:388: UserWarning: These node types are not in the graph: {'ReferenceUsage'}.
  warn(f"These node types are not in the graph: {mismatched_node_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'ParameterMembership', 'FeatureValue', 'ReturnParameterMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


1

In [10]:
roll_up_multiplicity(lpg, helper_client.elements_by_id['eb96afae-0f09-4912-861e-705bb33a4202'], 'upper')

1

In [11]:
roll_up_multiplicity(lpg, helper_client.elements_by_id['648fd9b3-2a6e-40e8-b3e9-b8ebb407ce07'], 'upper')

1

In [12]:
roll_up_multiplicity(lpg, helper_client.elements_by_id['4f446a59-bc83-4285-aa2b-6c5dd9bd5541'], 'upper')

1

In [13]:
banded_featuring_graph = lpg.get_projection("Expanded Banded Graph")

In [14]:
banded_roots = [
        node
        for node in banded_featuring_graph.nodes
        if banded_featuring_graph.out_degree(node) < 1
    ]
banded_roots

['eb96afae-0f09-4912-861e-705bb33a4202']

In [15]:
from pymbe.interpretation.interp_playbooks import build_expression_sequence_templates
from pymbe.label import get_label_for_id

In [16]:
expr_seqs = build_expression_sequence_templates(lpg)
for seq in expr_seqs:
    sig_seq = []
    for item in seq:
        sig_seq.append(get_label_for_id(item, helper_client.elements_by_id))
    print(sig_seq)

In [17]:
from pymbe.query.metamodel_navigator import feature_multiplicity

In [18]:
feature_multiplicity(helper_client.elements_by_id[connection_id], helper_client.elements_by_id, "upper")

1

In [19]:
from pymbe.interpretation.interp_playbooks import build_sequence_templates
build_sequence_templates(lpg)

[['eb96afae-0f09-4912-861e-705bb33a4202',
  '4f446a59-bc83-4285-aa2b-6c5dd9bd5541',
  'babeb025-52f3-4ebe-9f0f-0414a4b6240d'],
 ['eb96afae-0f09-4912-861e-705bb33a4202',
  '648fd9b3-2a6e-40e8-b3e9-b8ebb407ce07',
  '7328c370-26d7-40e4-9a36-c88ef76c7d30'],
 ['eb96afae-0f09-4912-861e-705bb33a4202',
  'c109629e-dbf1-4708-bba7-63c8cb647484']]

In [20]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake
)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'ParameterMembership', 'ResultExpressionMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'Superclassing'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [21]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

['Power Group: Part', 'Power Source: Part', 'Power Out: Port'], 1
['Power Group: Part', 'Power User: Part', 'Power In: Port'], 1
['Power Group: Part', 'powerToUser: Connection'], 1


## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

Show all interpretation sequence sets (limited to length of 5).

In [22]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)

Port, id = f29c8a2e-a7d3-4685-bfbb-3d54b5dd704c, size = 2
[Port#0]
[Port#1]
Connection, id = 9739035d-8c51-4eab-8ab9-ed703a21f1c4, size = 1
[Connection#0]
Part, id = a1eb8672-fdc4-446e-a004-861974660152, size = 3
[Part#0]
[Part#1]
[Part#2]
Power Group: Part, id = eb96afae-0f09-4912-861e-705bb33a4202, size = 1
[Part#0]
Power Source: Part, id = 4f446a59-bc83-4285-aa2b-6c5dd9bd5541, size = 1
[Part#0, Part#1]
Power Out: Port, id = babeb025-52f3-4ebe-9f0f-0414a4b6240d, size = 1
[Part#0, Part#1, Port#0]
Power User: Part, id = 648fd9b3-2a6e-40e8-b3e9-b8ebb407ce07, size = 1
[Part#0, Part#2]
Power In: Port, id = 7328c370-26d7-40e4-9a36-c88ef76c7d30, size = 1
[Part#0, Part#2, Port#1]
powerToUser: Connection, id = c109629e-dbf1-4708-bba7-63c8cb647484, size = 1
[Part#0, Connection#0]
15e0e4a3-7a35-4495-8111-cbe004f6fdda «Feature», id = 15e0e4a3-7a35-4495-8111-cbe004f6fdda, size = 1
[Part#0, Connection#0, Part#0, Part#1, Port#0]
7c8c4bb7-8f20-41a9-947e-978eb36fc0ff «Feature», id = 7c8c4bb7-8f20-41a